In [1]:
import sys
sys.path.append(r'D:\CE7454_2018\VAE FOR STOCHASTIC INTEGER PROGRAMMING\CVAE for stochastic programming')
from local_stochoptim.stochprob.facility_location import facility_location_problem as flp

In [2]:
import matplotlib.pyplot as plt
# %matplotlib notebook
# import stochoptim.stochprob.facility_location.facility_location_problem as flp
import stochoptim.stochprob.facility_location.facility_location_solution as fls
import stochoptim.stochprob.facility_location.facility_location_uncertainty as flu
import stochoptim.scengen.scenario_tree as st
# for clustering
from sklearn_extra.cluster import KMedoids
import numpy as np

In [3]:
import os
import time
import torch
import argparse
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset, TensorDataset
from collections import defaultdict

# for VAE
# from models_gcn import VAE

# for clustering
from sklearn_extra.cluster import KMedoids
import pickle

### Generate Problem Settings

In [4]:
# n_origins = 2
# n_destinations = 2
# n_intermediates = 40
# #=======================
# n_scenarios = 200
# distribution = 'uniform'
# lb = 5
# ub = 20
# corr = 0.4

n_facility_locations = 10
n_client_locations = 20
n_zones = 1
#----------------------------------------------------------------------------------------
n_scenarios = 200
p = 0.8

### No objectives

In [5]:
############################################################GCN
# from config import *
# config_path = "configs/tsp10.json"
# config = get_config(config_path)
# print("Loaded {}:\n{}".format(config_path, config))
# from models_gcn import VAE
# torch.manual_seed(0)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed(0)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # the net
# vae = VAE(
#     config=config,     
#     encoder_layer_sizes=[4, 128],
#     latent_size=2,
#     decoder_layer_sizes=[128, 4],
#     conditional=True,
#     num_labels=2 if True else 0).to(device)

# # Specify a path
# PATH = "trained_nets/saved_model_GCN_"+ str(100) + ".pt"
# # load parameters
# vae.load_state_dict(torch.load(PATH))
# vae.eval()

#######################################################
# for VAE
from models_gcn import VAE
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from config import *
config_path = "configs/tsp50_ori.json"
config = get_config(config_path)
print("Loaded {}:\n{}".format(config_path, config))
# the net
vae = VAE(
    config=config,     
    encoder_layer_sizes=[20, 256, 128],
    latent_size=2,
    decoder_layer_sizes=[128, 256, 20],
    conditional=True,
    num_labels=2 if True else 0).to(device)


# Specify a path
PATH = "trained_nets/1saved_model_GCN_"+ str(100) + ".pt"
# load parameters
vae.load_state_dict(torch.load(PATH))
vae.eval()

Loaded configs/tsp50_ori.json:
{'expt_name': 'tsp50', 'gpu_id': '0', 'train_filepath': './data/tsp50_train_concorde.txt', 'val_filepath': './data/tsp50_val_concorde.txt', 'test_filepath': './data/tsp50_test_concorde.txt', 'num_nodes': 50, 'num_neighbors': -1, 'node_dim': 64, 'voc_nodes_in': 2, 'voc_nodes_out': 2, 'voc_edges_in': 3, 'voc_edges_out': 2, 'beam_size': 1280, 'hidden_dim': 128, 'num_layers': 10, 'mlp_layers': 3, 'aggregation': 'mean', 'max_epochs': 1500, 'val_every': 5, 'test_every': 100, 'batch_size': 20, 'batches_per_epoch': 500, 'accumulation_steps': 1, 'learning_rate': 0.001, 'decay_rate': 1.01}
CUDA available, using GPU ID 0


VAE(
  (encoder): Encoder(
    (linear_means): Linear(in_features=128, out_features=2, bias=True)
    (linear_log_var): Linear(in_features=128, out_features=2, bias=True)
  )
  (decoder): Decoder(
    (MLP): Sequential(
      (L0): Linear(in_features=4, out_features=128, bias=True)
      (A0): ReLU()
      (L1): Linear(in_features=128, out_features=256, bias=True)
      (A1): ReLU()
      (L2): Linear(in_features=256, out_features=20, bias=True)
      (sigmoid): Sigmoid()
    )
    (linear_dim_red): Linear(in_features=128, out_features=2, bias=True)
  )
  (GCN): ResidualGatedGCNModel(
    (nodes_coord_embedding): Linear(in_features=64, out_features=128, bias=False)
    (edges_values_embedding): Linear(in_features=1, out_features=64, bias=False)
    (edges_embedding): Embedding(3, 64)
    (gcn_layers): ModuleList(
      (0): ResidualGatedGCNLayer(
        (node_feat): NodeFeatures(
          (U): Linear(in_features=128, out_features=128, bias=True)
          (V): Linear(in_features=128,

### With objectives

In [6]:
################################################### GCN
from config import *
config_path = "configs/tsp50_ori.json"
config = get_config(config_path)
print("Loaded {}:\n{}".format(config_path, config))
from models_objs_gcn import VAE
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# the net
vae_objs = VAE(
    config=config,     
    encoder_layer_sizes=[20, 256, 128],
    latent_size=2,
    decoder_layer_sizes=[128, 256, 20],
    conditional=True,
    num_labels=2 if True else 0).to(device)

# Specify a path
PATH = "trained_nets/1ob_saved_model_GCN_dependence_"+ str(300) + ".pt"
# load parameters
vae_objs.load_state_dict(torch.load(PATH))
vae_objs.eval()

Loaded configs/tsp50_ori.json:
{'expt_name': 'tsp50', 'gpu_id': '0', 'train_filepath': './data/tsp50_train_concorde.txt', 'val_filepath': './data/tsp50_val_concorde.txt', 'test_filepath': './data/tsp50_test_concorde.txt', 'num_nodes': 50, 'num_neighbors': -1, 'node_dim': 64, 'voc_nodes_in': 2, 'voc_nodes_out': 2, 'voc_edges_in': 3, 'voc_edges_out': 2, 'beam_size': 1280, 'hidden_dim': 128, 'num_layers': 10, 'mlp_layers': 3, 'aggregation': 'mean', 'max_epochs': 1500, 'val_every': 5, 'test_every': 100, 'batch_size': 20, 'batches_per_epoch': 500, 'accumulation_steps': 1, 'learning_rate': 0.001, 'decay_rate': 1.01}
CUDA available, using GPU ID 0


VAE(
  (encoder): Encoder(
    (linear_means): Linear(in_features=128, out_features=2, bias=True)
    (linear_log_var): Linear(in_features=128, out_features=2, bias=True)
    (obj): Sequential(
      (L0): Linear(in_features=128, out_features=128, bias=True)
      (A0): ReLU()
    )
    (linear_obj): Linear(in_features=128, out_features=1, bias=True)
    (m_obj): Sigmoid()
  )
  (decoder): Decoder(
    (MLP): Sequential(
      (L0): Linear(in_features=4, out_features=128, bias=True)
      (A0): ReLU()
      (L1): Linear(in_features=128, out_features=256, bias=True)
      (A1): ReLU()
      (L2): Linear(in_features=256, out_features=20, bias=True)
      (sigmoid): Sigmoid()
    )
    (linear_dim_red): Linear(in_features=128, out_features=2, bias=True)
  )
  (GCN): ResidualGatedGCNModel(
    (nodes_coord_embedding): Linear(in_features=64, out_features=128, bias=False)
    (edges_values_embedding): Linear(in_features=1, out_features=64, bias=False)
    (edges_embedding): Embedding(3, 64)


### Generate instances

In [7]:
num_instances = 100

In [8]:
# param_list = []
# scenario_tree_list = []
# for _ in range(num_instances):    

#     param = flp.generate_random_parameters(n_facility_locations, n_client_locations, n_zones)
   
#     client_uncertainty = flu.ClientsPresence(n_scenarios, n_client_locations, lb=0.4, ub=0.6)   
    
#     scenario_tree = client_uncertainty.get_scenario_tree()    

#     param_list.append(param)
#     scenario_tree_list.append(scenario_tree)
# df=open('test_data_120.txt', 'wb')
# pickle.dump(param_list, df, -1)
# pickle.dump(scenario_tree_list, df, -1)
# df.close()    

In [9]:
dfr=open('test_data.txt', 'rb')
param_list = pickle.load(dfr)
scenario_tree_list = pickle.load(dfr)

In [10]:
#Define number of clusters
n_c = 20

our_objs = []
our_objs_objs = []
km_objs = []
optim = []

our_objs_t = []
our_objs_objs_t = []
km_objs_t = []
optim_t = []
for i in range(num_instances):
    # Instantiate Problem
    param = param_list[i]
#     param['penalty'] = np.array([100., 100., 100., 100., 100., 100., 100., 100., 100., 100.])
    facility_problem = flp.FacilityLocationProblem(param)
    scenario_tree = scenario_tree_list[i]   
    ############################################################ 1. Clustering with nets
    pos_c = param['pos_client']
    pos_f = param['pos_facility']
    open_cost = param['opening_cost']/81.
    fac_cap = param['facility_capacity']/60.  
    ##### little batch due to memory 
    ite = 2
    bs = int(n_scenarios/ite)
    tensor_c_fn = torch.from_numpy(np.concatenate((pos_f,open_cost[:,None],fac_cap[:,None]),axis=1))
    tensor_c_fn = tensor_c_fn.unsqueeze(0).repeat(bs, 1, 1)
    tensor_c_cn = torch.from_numpy(pos_c)
    tensor_c_cn = tensor_c_cn.unsqueeze(0).repeat(bs, 1, 1)    
    
    with torch.no_grad():
        s_t = time.time()
        z_list = []
        lat_list = []        
        for ii in range(ite): 
            tensor_x = torch.from_numpy(scenario_tree.to_numpy()[ii*bs:(ii+1)*bs])
            ### 10 can be changed to any value > 1
            tensor_x = tensor_x.unsqueeze(1).repeat(1, 2, 1)
            x, y_cn, y_fn, y_dist = tensor_x.to(device).float(), tensor_c_cn.to(device).float(), tensor_c_fn.to(device).float(), None  
            recon_x, mean, log_var, z, target, _, lat = vae(x, y_cn=y_cn, y_fn=y_fn, y_dist=y_dist)
            z_list.append(z)
            lat_list.append(lat)            
        z = torch.cat(z_list)
        lat = torch.cat(lat_list)
        
        X = z.detach().cpu().numpy()
#         X = lat.detach().cpu().numpy()  
        kmedoids = KMedoids(n_clusters=n_c, random_state=0).fit(X)
        repres =  kmedoids.cluster_centers_
        indices= []
        for j in range(repres.shape[0]):
            indices.append(np.where(np.all(X==repres[j], axis=1))[0][0])
        unique_elements, counts_elements = np.unique(kmedoids.labels_, return_counts=True)
        if kmedoids.predict(repres)[0]!=unique_elements[0]:
            counts_elements=counts_elements[::-1]
        print(counts_elements) 
        if len(counts_elements)<n_c:
            continue
        clustering_nets = st.twostage_from_scenarios(scenarios=scenario_tree.to_numpy()[np.array(indices)], 
                                                     n_rvar={'h': n_client_locations},
                                                     weights=np.array(counts_elements/counts_elements.sum()))
        solution_clustering_nets = facility_problem.solve(clustering_nets, verbose=0)
        our_objs_t.append(time.time()-s_t)        
        stage_0_decisions_nets = solution_clustering_nets.decision_process_fixed({0: ['x']})
        solution_nets = facility_problem.solve(scenario_tree, 
                                         decision_process=stage_0_decisions_nets, verbose=0)
        our_objs.append(solution_nets.objective_value)   
    
    ###################################################################### 2. Clustering with nets with objectives
        s_t = time.time()    
#         recon_x, mean, log_var, z, target, objs, _, lat = vae_objs(x, y)
#         X = torch.cat((z,objs),1).detach().cpu().numpy()
        
        z_list = []
#         lat_list = []        
        for ii in range(ite): 
            tensor_x = torch.from_numpy(scenario_tree.to_numpy()[ii*bs:(ii+1)*bs])
            ### 10 can be changed to any value > 1
            tensor_x = tensor_x.unsqueeze(1).repeat(1, 2, 1)
            x, y_cn, y_fn, y_dist = tensor_x.to(device).float(), tensor_c_cn.to(device).float(), tensor_c_fn.to(device).float(), None  
            recon_x, mean, log_var, z, target, objs, _, lat = vae_objs(x, y_cn=y_cn, y_fn=y_fn, y_dist=y_dist)
            Xz = torch.cat((z,objs),1)
#             Xlat = torch.cat((lat,objs),1)            
            z_list.append(Xz)
#             lat_list.append(Xlat)            
        z = torch.cat(z_list)
#         print(z_list)
#         lat = torch.cat(lat_list)
        
#         X = lat.detach().cpu().numpy()   
        X = z.detach().cpu().numpy()           
        kmedoids = KMedoids(n_clusters=n_c, random_state=0).fit(X)
        repres =  kmedoids.cluster_centers_
        indices= []
        for j in range(repres.shape[0]):
            indices.append(np.where(np.all(X==repres[j], axis=1))[0][0])
        unique_elements, counts_elements = np.unique(kmedoids.labels_, return_counts=True)
        if kmedoids.predict(repres)[0]!=unique_elements[0]:
            counts_elements=counts_elements[::-1]
        print(counts_elements)
        if len(counts_elements)==1:
            our_objs.pop()
            continue        
        clustering_nets = st.twostage_from_scenarios(scenarios=scenario_tree.to_numpy()[np.array(indices)], 
                                                     n_rvar={'h': n_client_locations},
                                                     weights=np.array(counts_elements/counts_elements.sum()))        
        solution_clustering_nets = facility_problem.solve(clustering_nets, verbose=0)        
        our_objs_objs_t.append(time.time()-s_t)
        stage_0_decisions_nets = solution_clustering_nets.decision_process_fixed({0: ['x']})
        solution_nets = facility_problem.solve(scenario_tree, 
                                         decision_process=stage_0_decisions_nets, verbose=0)        
        our_objs_objs.append(solution_nets.objective_value)       
    ############################################################################## 3. start k-medoids
    s_t = time.time()    
    KX = scenario_tree.to_numpy()
    Kkmedoids = KMedoids(n_clusters=n_c, random_state=0).fit(KX)
    Krepres =  Kkmedoids.cluster_centers_
    Kindices= []
    for j in range(Krepres.shape[0]):
        Kindices.append(np.where(np.all(KX==Krepres[j], axis=1))[0][0])
    Kunique_elements, Kcounts_elements = np.unique(Kkmedoids.labels_, return_counts=True)  
    if Kkmedoids.predict(Krepres)[0]!=Kunique_elements[0]:
        Kcounts_elements=Kcounts_elements[::-1]
    print(Kcounts_elements) 
    if len(Kcounts_elements)<n_c:
        our_objs.pop()
        our_objs_objs.pop()
        continue      
    Kclustering_nets = st.twostage_from_scenarios(scenarios=scenario_tree.to_numpy()[np.array(Kindices)], 
                                                     n_rvar={'h': n_client_locations},
                                                     weights=np.array(Kcounts_elements/Kcounts_elements.sum()))
    Ksolution_clustering_nets = facility_problem.solve(Kclustering_nets, verbose=0)
    km_objs_t.append(time.time()-s_t)    
    Kstage_0_decisions_nets = Ksolution_clustering_nets.decision_process_fixed({0: ['x']})
    Ksolution_nets = facility_problem.solve(scenario_tree, 
                                         decision_process=Kstage_0_decisions_nets, verbose=0)
    km_objs.append(Ksolution_nets.objective_value)
    ############################################################################### 4. actual solutions
#     s_t = time.time()     
#     actual_solution = facility_problem.solve(scenario_tree, verbose=0, timelimit=3600)
#     optim_t.append(time.time()-s_t)       
#     optim.append(actual_solution.objective_value)
    print(str(i+1)+" instance(s) solved")

[ 1  3  5 14  5 14 10 11 11 20 17 14 13  2  3  9 12  5 11 20]
[ 1 16  3 14  3  7 17  5 11  1  1 13 11 14 21 11 20  7 13 11]
[11 18 11 18 13 13  8  9  5 12  9  7 10  9  8  5  4 11 10  9]
1 instance(s) solved
[ 1 11 16  4  7  2  2 11  2 10  3 21 14 15 13 15 10  7 19 17]
[ 6  8  2  1 14 17 12  7 11  3 14 15 13 13 16  8 15  8 10  7]
[24 19 11  7  8 12 11  7  7 12  3 12 11 15  5  8  5  4 11  8]
2 instance(s) solved
[26 17  5  4  2  5  8  1  2  1  2  6  8 13  9 15 20 18 20 18]
[13  4 17 19  1  8 15 12  2 11  9 17  2 18  4  2 10 11 10 15]
[26 11 26 10 15  8  9  6  4  9 15  9 11  6  4  6 10  3  4  8]
3 instance(s) solved
[ 2  2  9 25  1  8 12  3  2 19  2 17  5  8 16  9  4 18 19 19]
[14 15  2  1  4  3 10 18  2  9  9 16 17  3  9 17 16 15 10 10]
[23 10  6 12 11 15  5 12 18  5 15 11  5  6  7  8 10  6  7  8]
4 instance(s) solved
[ 8  7 16  6  4 14  4 13 16 15  9 14 10  8  5  5  8 14 14 10]
[ 5  3  9  7  6 13  9 14 24 10 11  6  4 13 11 12 18 11  5  9]
[17 21  6 11 15 12 14  7  7 13 10 12  7 14  3  3

[24 12  8  9 11 24  7 19 14 11  8  3  3  5 11 10  1  2  7 11]
40 instance(s) solved
[ 4 10 11  8  8  7  5 16  6  1  7 15 17 10  9  9 21 19  8  9]
[16  5 15  3 11  4  5 10 16  8 14  7  9 11  9  9 10  8 19 11]
[24  9 20  6 10 15 12 12  5  3 10  8  6 10 14  5  8 10  6  7]
41 instance(s) solved
[11 10 19 16 10  4  8 16  3  8  2  4  7 10 17 15  8  7 12 13]
[ 2  4  8  4  3  2 14 16  9 18 15 11 11  9  9  8 12 16 11 18]
[12 20 10 20 12  6 10  5 13 10 10  8 11  9  3 11  7  8 12  3]
42 instance(s) solved
[ 5 19  2  1  9  5 13 17  3  8  8  7 18 21 10  8 13  6 19  8]
[10  8  9  8 18 11 12  6 10 22 16  2  1  5  8 10  1 12 18 13]
[22 18  8 10 17 13  6 11  9  7 10 13  9  7  6  1 14  7  5  7]
43 instance(s) solved
[11  7  5 16  9 14 12  6  2  4 16  5  8 12 12  7  1 14 23 16]
[ 4  5 14 12 12 14  1  9 10  6 18 15  6  6  1 12 12 16 10 17]
[13 15 12 11 20 15 20 12 11  7  5 12  8 12  8  4  4  4  5  2]
44 instance(s) solved
[ 7 20  3 13  4  7  7  5 10 11 10 11 11  8  6 16 11 16 10 14]
[11  8  2 22  3  3 15 

[14 12  5 12 14 15  1  7 14 13  5  2 10  6 11 13  9 19 11  7]
[ 1 10  7 15  2  2 14 13 11 14 13 15  4 11  7  5 13 11 18 14]
[12 17 15  9 20  7 11 11  9  5 14  3  9 11  3  4  8 12  8 12]
80 instance(s) solved
[13 20  2 14 17  1 16  2  2 15 10 15  7  4  4 12  2 16  8 20]
[ 6 17  2  2  1 11  7 13  9  6  9 19  7 24 14 13  9 10 10 11]
[23 14 13 12 14  9  9 10  6 10  7  8  6 11  8  9  3  2  8 18]
81 instance(s) solved
[ 3  1 15  4  6  4 15  7  2  8  6 12 13 16  9 21 17 14 11 16]
[ 3 14  8 17  1  1  2  5  5  4 13 12 10 19  7 12 21 17 18 11]
[10 21 14 14 14  8  8 11  7 11 14  3  8 11 11  7  7  7  6  8]
82 instance(s) solved
[23  3 12  2  1  2  1 22  8  5 12 16 16 18  8  9  8  9  9 16]
[13  6  1  4  5  2  6 11 21  2  7 10 16 13 19 11 18 10 14 11]
[13 15 15  8 12 15 11 14  3  8 16  9  7 10  8  7  3 12  7  7]
83 instance(s) solved
[ 2 14  7 13 17  3  5 12  4  8  9 11 14 10  8 13 15  7 15 13]
[ 4 22  2 26  5  5  1  5 12  3  2  1 12 12 21 14 14 13 12 14]
[25 14  8  9  8 16 12  8 12 11  9  9  7  7  

C:\Users\WUYAOXIN\AppData\Local\Continuum\miniconda3\envs\deeplearn_course\lib\site-packages\sklearn_extra\cluster\_k_medoids.py:279: UserWarning: Cluster 12 is empty! self.labels_[self.medoid_indices_[12]] may not be labeled with its corresponding cluster (12).
  "its corresponding cluster ({k}).".format(k=k)
C:\Users\WUYAOXIN\AppData\Local\Continuum\miniconda3\envs\deeplearn_course\lib\site-packages\sklearn_extra\cluster\_k_medoids.py:279: UserWarning: Cluster 12 is empty! self.labels_[self.medoid_indices_[12]] may not be labeled with its corresponding cluster (12).
  "its corresponding cluster ({k}).".format(k=k)
C:\Users\WUYAOXIN\AppData\Local\Continuum\miniconda3\envs\deeplearn_course\lib\site-packages\sklearn_extra\cluster\_k_medoids.py:279: UserWarning: Cluster 12 is empty! self.labels_[self.medoid_indices_[12]] may not be labeled with its corresponding cluster (12).
  "its corresponding cluster ({k}).".format(k=k)


[15 16 16 11 10 14  9 16 11 16  8  7 12  6  9 10  8  3  3]
[ 8  6 11  3 11  1  7 14  2  3  6 17  7 12 14 15 11 13 16 23]
[ 2  7  4  4  1  7  8  8  7 11 10  8 25 12 10 11  8 17 18 22]
[20 16 14 13 12 12 11  5 14  4  9  8  7  3 12  7  7  7 12  7]
97 instance(s) solved
[ 9  4  9  7 11  5 10  8  2  9  7 14 11 11 11 16 11 17 15 13]
[12  1 12  8  9  9  7  7  9  4 15 14 10 10 11  1 13 17  7 24]
[20 10 16 12 16 13 15 15 13  4  3  4 10  4 12  7  6  3  4 13]
98 instance(s) solved
[ 3 11 17  4  2  2  7  2 15  2 14  1  7  1 21 19 13 28 14 17]
[ 1 14  1  8  1  3 24  2 16  1  1  8 13 13 12  6 15 15 24 22]
[17 23 10 11 11  9 12  7 14  9 15  2  9  8  9 12  4  9  6  3]
99 instance(s) solved
[ 3  2 10  2  2  4 18 10 16 11 18  6  4 13  5 17 15 18  8 18]
[ 5  9  9 15 14  4 11  5 11  4 13 16  1  9  7  1 12 18 21 15]
[20 14 13 15 13 17 16  6  6  4  6 11  2  7 11 11  6  9  7  6]
100 instance(s) solved


In [13]:
optim = np.array([187.23003644, 206.15751885, 224.5245098 , 191.40149193,
       239.40473225, 206.5043535 , 206.67589764, 211.7566585 ,
       197.73466796, 186.27781237, 233.08182302, 215.33900628,
       259.45675712, 197.96929771, 221.31105653, 209.81325552,
       201.65517774, 277.87564305, 170.65501124, 237.51704056,
       234.87342333, 215.48142672, 223.31909969, 218.03899513,
       197.2601914 , 252.09315303, 191.22903326, 240.51755695,
       179.16930743, 197.42881953, 205.52694304, 230.92310521,
       184.39006507, 201.85711007, 195.42997752, 182.533563  ,
       218.21735432, 186.14317332, 198.29664614, 207.88999129,
       220.37075498, 254.42472625, 188.09644734, 221.56808222,
       204.29960202, 225.46639032, 192.85706266, 214.09278741,
       229.05229188, 211.90821175, 209.18169235, 210.9864359 ,
       211.80094388, 197.0825418 , 265.76734749, 193.78035634,
       210.33739346, 209.62050458, 209.5004609 , 229.75316114,
       222.6034975 , 214.40686593, 232.43437988, 222.14639206,
       215.26770203, 240.06629518, 201.61772357, 221.77150757,
       272.1483901 , 212.71407976, 182.58705997, 203.58569985,
       235.0882005 , 219.03366409, 199.61899629, 226.35419549,
       203.22792331, 195.94474362, 211.07924656, 199.59405911,
       217.01346873, 253.20719397, 200.90400934, 193.19327343,
       202.84568747, 200.57815868, 269.08032225, 202.39132566,
       203.02280469, 224.25568621, 230.10193593, 303.51080187,
       197.27544187, 291.22149172, 273.90772579, 201.30817578,
       225.03184452, 250.27336866, 210.42611322, 194.57402473])

In [14]:
((np.array(our_objs)-np.array(optim))/np.array(optim)).mean()  ####1000 with gcn

0.48493800445567353

In [15]:
((np.array(km_objs)-np.array(optim))/np.array(optim)).mean()   ####1000 

16.14660218727941

In [16]:
((np.array(our_objs_objs)-np.array(optim))/np.array(optim)).mean()   ####1000 with gcn

0.5258573698692721

In [17]:
np.array(our_objs).mean(), np.array(our_objs).std()

(324.1497363815068, 238.3958636664788)

In [18]:
np.array(km_objs).mean(), np.array(km_objs).std()

(3734.710356586069, 4255.40915455133)

In [19]:
np.array(our_objs_objs).mean(), np.array(our_objs_objs).std()

(335.8329318757474, 266.3562801208688)

In [20]:
np.array(optim).mean(), np.array(optim).std()

(216.4344765579798, 24.81212342594604)

In [21]:
np.array(our_objs_t).mean()

1.0317539858818054

In [20]:
np.array(km_objs_t).mean()

0.16343451738357545

In [21]:
np.array(our_objs_objs_t).mean()

0.5651468205451965

In [22]:
np.array(optim_t).mean()

C:\Users\WUYAOXIN\AppData\Local\Continuum\miniconda3\envs\deeplearn_course\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice.
  """Entry point for launching an IPython kernel.
C:\Users\WUYAOXIN\AppData\Local\Continuum\miniconda3\envs\deeplearn_course\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan

In [34]:
np.array(optim)

array([452.91286378, 474.06089563, 457.67697744, 456.27392112,
       470.35218918, 452.63253975, 444.04374733, 473.35251714,
       509.96027779, 437.44504018, 455.01461076, 473.12787913,
       490.81376567, 487.67546841, 447.95142432, 469.1113199 ,
       491.76059451, 503.28941237, 465.5081108 , 479.37039468])